In [ ]:

import pandas as pd
import numpy as np
import lightgbm as lgb
import gc
import os
import sys

# ==============================================================================
# 1. CHARGEMENT ET PRÉPARATION
# ==============================================================================
print("Chargement des données...")
# Assure-toi que le fichier est bien celui généré par ton script de processing
data = pd.read_csv('data/processed_data.csv', parse_dates=['date'])

# --- LOG-TRANSFORMATION DE LA CIBLE ---
# On applique log1p sur les ventes d'entraînement pour la métrique RMSLE
data.loc[data['is_train'] == 1, 'sales'] = np.log1p(data.loc[data['is_train'] == 1, 'sales'])

# Suppression de la colonne transactions brute si elle existe (on utilise les lags)
if 'transactions' in data.columns:
    data.drop(columns=['transactions'], inplace=True)

# Conversion en type 'category' pour LightGBM (Optimisation mémoire et vitesse)
# Note : On ne met PAS 'family' ici car on va boucler dessus, elle ne sera pas une feature
categorical_features = [
    "store_nbr", "city", "state", "type", "cluster",
    "is_holiday", "dayofweek", "month"
]

for col in categorical_features:
    if col in data.columns:
        data[col] = data[col].astype('category')

# Liste des features (On exclut 'family' car c'est notre boucle, et les IDs/Dates)
features = [col for col in data.columns if col not in ['id', 'sales', 'is_train', 'date', 'family']]

print(f"Features utilisées : {len(features)}")
print(features)

# Liste des familles uniques
FAMILIES = data['family'].unique()
print(f"Nombre de modèles à entraîner : {len(FAMILIES)}")

# ==============================================================================
# 2. BOUCLE D'ENTRAÎNEMENT PAR FAMILLE
# ==============================================================================
all_predictions = []
test_ids_global = []

# Paramètres LightGBM (Basés sur ta baseline précédente)
lgb_params = {
    'objective': 'regression_l1', # Ou 'regression' (mse), mais l1 est souvent robuste
    'metric': 'rmse',
    'n_estimators': 1500,         # Un peu moins que le global car moins de données par famille
    'learning_rate': 0.02,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'lambda_l1': 0.1,
    'lambda_l2': 0.1,
    'num_leaves': 31,
    'verbose': -1,
    'n_jobs': -1,
    'seed': 42,
    'boosting_type': 'gbdt',
}

print("\nDébut de l'entraînement par famille...")

for i, fam in enumerate(FAMILIES):
    print(f"\n[{i+1}/{len(FAMILIES)}] Traitement de la famille : {fam}")
    
    # 1. Filtrer les données pour cette famille uniquement
    df_fam = data[data['family'] == fam].copy()
    
    # 2. Séparation Train / Test
    train_df = df_fam[df_fam['is_train'] == 1]
    test_df = df_fam[df_fam['is_train'] == 0]
    
    # 3. Création du jeu de validation (15 derniers jours du train de cette famille)
    last_train_date = train_df['date'].max()
    validation_start_date = last_train_date - pd.DateOffset(days=15)
    
    valid_mask = train_df['date'] >= validation_start_date
    train_mask = train_df['date'] < validation_start_date
    
    X_train = train_df.loc[train_mask, features]
    y_train = train_df.loc[train_mask, 'sales']
    
    X_valid = train_df.loc[valid_mask, features]
    y_valid = train_df.loc[valid_mask, 'sales']
    
    X_test = test_df[features]
    ids_test = test_df['id'] # On garde les IDs pour la fin
    
    # 4. Entraînement du modèle
    model = lgb.LGBMRegressor(**lgb_params)
    
    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric='rmse',
        callbacks=[
            lgb.log_evaluation(period=0), # 0 pour ne pas spammer la console
            lgb.early_stopping(stopping_rounds=50)
        ],
        categorical_feature=[c for c in categorical_features if c in features]
    )
    
    # Affichage du score de validation pour cette famille
    best_score = model.best_score_['valid_0']['rmse']
    print(f"   -> Best Validation RMSE: {best_score:.4f}")
    
    # 5. Prédiction
    preds_log = model.predict(X_test, num_iteration=model.best_iteration_)
    
    # Inverse Log (expm1) et correction des négatifs
    preds = np.expm1(preds_log)
    preds[preds < 0] = 0
    
    # Stockage des résultats
    fam_submission = pd.DataFrame({'id': ids_test, 'sales': preds})
    all_predictions.append(fam_submission)
    
    # Nettoyage mémoire immédiat
    del df_fam, train_df, test_df, X_train, X_valid, X_test, model
    gc.collect()

# ==============================================================================
# 3. ASSEMBLAGE FINAL
# ==============================================================================
print("\nAssemblage de toutes les prédictions...")

# Concaténer tous les dataframes de chaque famille
submission_df = pd.concat(all_predictions)

# Trier par ID pour respecter l'ordre de Kaggle (Très important)
submission_df = submission_df.sort_values('id').reset_index(drop=True)

# Sauvegarde
filename = 'submission_lgbm_per_family.csv'
submission_df.to_csv(filename, index=False)

print(f"\nFichier '{filename}' créé avec succès !")
print(submission_df.head())

# Vérification rapide de la cohérence (pas de trous)
expected_len = 28512 # Taille standard du test set Store Sales
if len(submission_df) == expected_len:
    print(f"Taille du fichier correcte : {len(submission_df)} lignes.")
else:
    print(f"ATTENTION : Taille incorrecte ({len(submission_df)} au lieu de {expected_len})")